# FANS Dashboard
This Jupyter Notebook is designed to post-process, interpret, and visualize results generated by FANS. The notebook provides tools for exploring the hierarchical structure of HDF5 files, extracting and summarizing simulation data, and preparing the results for visualization in ParaView.

## Index of Contents

1. [Prerequisites](#prerequisites)
2. [Importing Required Modules](#importing-required-modules)
3. [Load and Explore Simulation Results](#step-1-load-and-explore-simulation-results)
4. [Specify and Extract Data for Analysis](#step-2-specify-and-extract-data-for-analysis)
5. [Compute Tensor Measures](#step-3-compute-tensor-measures)
6. [Post-processing and Writing Data to HDF5](#step-4-post-processing-and-writing-data-to-hdf5)
7. [Visualizing Basic Data](#step-5-visualizing-basic-data)
8. [Exporting Data to XDMF for Visualization](#step-6-exporting-data-to-xdmf-for-visualization)
9. [Summary](#summary)

## Importing Required Modules
These modules provide essential functions for data extraction, post-processing, and visualization.

In [ ]:
import numpy as np
from MSUtils.fans_dashboard.utils import *
from MSUtils.fans_dashboard.postprocessing import compute_rank2tensor_measures
from MSUtils.general.h52xdmf import write_xdmf
from MSUtils.fans_dashboard.plotting import plot_subplots

# Usage

## Step 1: Load and Explore Simulation Results
To begin working with your FANS simulation results, first load the data from an HDF5 file. You can specify the path to your results file via `file_path`. 

Before diving into analysis or visualization, it’s crucial to understand the structure of your data. This can be done using the `identify_hierarchy` function. This function traverses the HDF5 file and returns a dictionary that represents the hierarchical structure of the data, including microstructures, load cases, and time steps. Each dataset's name and shape are captured, giving you a detailed map of the contents of the file.

After identifying the structure, you can summarize it to get a quick overview of what’s inside your HDF5 file using `summarize_hierarchy`. This function prints a summary to the console, including the number of microstructures, load cases, time steps, and available quantities for each load case. 
<!-- For example, you might see output like this:
```
Found 1 microstructure(s):
  Microstructure '/sphere/32x32x32/ms' has 1 load case(s):
    Load case 'load0' has 51 time step(s) and the following quantities:
      - absolute_error with shape (1,)
      - displacement with shape (32, 32, 32, 3)
      - microstructure with shape (32, 32, 32, 1)
      - strain with shape (32, 32, 32, 6)
      - strain_average with shape (6,)
      - stress with shape (32, 32, 32, 6)
      - stress_average with shape (6,)
``` -->

This summary helps you quickly understand the dataset without manually exploring the file, making it easier to decide which data to analyze or visualize.

In [ ]:
# Example usage
file_path = '../test/output/test_J2Plasticity.h5'

# Extract hierarchy information from the file
hierarchy = identify_hierarchy(file_path)
summarize_hierarchy(hierarchy)

## Step 2: Specify and Extract Data for Analysis
After exploring and summarizing the hierarchy of your simulation data, the next step is to specify which parts of the data you want to load for further analysis or visualization. This step involves selecting specific microstructures, load cases, quantities, and time steps.
This allows to narrow down the data you want to work with. Instead of loading the entire dataset, which could be large and cumbersome, you can focus on specific microstructures, load cases, quantities, and time steps that are relevant to your analysis.

The `extract_and_organize_data` function is used to extract the specified data from the HDF5 file and organize it into a structured format for further processing. Here’s what it does:

- Hierarchical Navigation: The function navigates through the hierarchical structure of the HDF5 file, which has already been identified by the `identify_hierarchy` function.

- Selection Criteria: It filters the data based on the user-specified microstructures, load cases, quantities, and time steps. Only the relevant data is extracted.

- Data Organization: The extracted data is organized into a nested dictionary structure. This structure is indexed by microstructure, load case, and quantity, with each dataset being stored as a NumPy array.

- Aggregation by Time Steps: The function also handles the time dimension, stacking data across different time steps along the 0th axis of the `NumPy` arrays. This makes it easier to process time-series data in subsequent steps.

- Returning Organized Data: Finally, the organized data is returned as a dictionary, where the keys correspond to microstructures, load cases, and quantities, and the values are the extracted data.


In [ ]:
# Specify which microstructures, load cases, and quantities to load
microstructures_to_load = list(hierarchy.keys())
load_cases_to_load = list(hierarchy[microstructures_to_load[0]].keys())
quantities_to_load = ['strain_average', 'stress_average', 'phase_stress_average_phase0', "stress"]
time_steps_to_load = []

# Extract the specified data, organized and sorted by time steps
data = extract_and_organize_data(file_path, hierarchy, 
                                 quantities_to_load, 
                                 microstructures_to_load, 
                                 load_cases_to_load, 
                                 time_steps_to_load)

strain_average = data[microstructures_to_load[0]][load_cases_to_load[0]]['strain_average']  
stress_average = data[microstructures_to_load[0]][load_cases_to_load[0]]['stress_average']
time_steps     = data[microstructures_to_load[0]][load_cases_to_load[0]]['time_steps']


## Step 3: Compute Tensor Measures
After selecting and organizing the data, various measures from the tensor data (e.g., stress or strain tensors) can be computed via `compute_rank2tensor_measures` function. This derives important scalar or vector measures of these Rank-2 tensors that may be crucial for interpreting the physical state of the simulated system.

- Input Data: The input to the function are symmetric Rank-2 tensors in Mandel notation ([e11, e22, e33, e12, e13, e23]), where the last dimension corresponds to the six independent components of the tensor.

- Output Data: The output is a dictionary where each entry corresponds to one of the computed measures, with the data organized in NumPy arrays that match the shape of the input tensors.

In [ ]:
# Specify measures to compute
measures_to_compute = ['von_mises', 'hydrostatic', 'deviatoric', 'principal', 
                       'max_shear', 'I_invariants', 'J_invariants', 'eigenvalues',
                       'eigenvectors', 'lode_angle']

# Compute the requested measures
results = compute_rank2tensor_measures(stress_average, measures_to_compute)

stress_average_deviatoric = results['deviatoric']

## Step 4: Post-processing and Writing Data to HDF5

The measures can be post-processed and written back to the HDF5 file via `postprocess_and_write_to_h5`. This allows to extend the original dataset with new, derived quantities that can be used for further analysis or visualization.

The `postprocess_and_write_to_h5` function handles the entire workflow of extracting data, processing it to compute the desired measures, and then writing the results back into the HDF5 file. Here's how it works:

- Data Extraction: The function first uses `extract_and_organize_data` to load the relevant data from the HDF5 file.

- Processing the Data: Once the data is extracted, the function computes the specified measures using the `compute_rank2tensor_measures` function.

- Naming Convention: The processed measures are saved under a new naming convention in the format quantity_measure (e.g., stress_average_von_mises). This makes it clear which original quantity each measure was derived from.

- Writing to HDF5: The function then writes the processed data back into the HDF5 file at the appropriate locations. The `write_processed_data_to_h5` helper function ensures that the data is stored correctly, using the appropriate group and dataset paths within the HDF5 file.

- Return Value: The function returns a dictionary containing the processed data, which can be used for further in-memory analysis.

In [ ]:
# Postprocessing and writing to h5 file
quantities_to_postprocess = ['stress_average', 'stress']
measures_to_postprocess = ['deviatoric', 'von_mises']
microstructures_to_postprocess = microstructures_to_load[0]
load_cases_to_postprocess = load_cases_to_load[0]

processed_data = postprocess_and_write_to_h5(file_path, hierarchy, quantities_to_postprocess, measures_to_postprocess, 
                                             microstructures_to_postprocess, load_cases_to_postprocess)

## Step 5: Visualizing basic data

After processing the data and computing the desired tensor measures, some basic measures can be visualized.

In [ ]:
# Plot stress average vs strain average for each component
fig = plot_subplots(strain_average, stress_average, labels_x=['Strain']*6, labels_y=['Stress']*6, 
                    subplot_titles=['component - 11', 'component - 22', 'component - 33', 'component - 12', 'component - 13', 'component - 23'], 
                    title='Stress average vs strain average', 
                    nrows=2, ncols=3, linewidth=1, markersize=4, linecolor=["blue"]*6, markercolor=["blue"]*6, fontsize=16)
fig.show()

In [ ]:
# Plot deviatoric stress average vs strain average for each component
fig = plot_subplots(strain_average, stress_average_deviatoric, labels_x=['Strain']*6, labels_y=['Stress_dev']*6,
                    subplot_titles=['component - 11', 'component - 22', 'component - 33', 'component - 12', 'component - 13', 'component - 23'],
                    title='Deviatoric stress average vs strain average',
                    nrows=2, ncols=3, linewidth=1, markersize=4, linecolor=["blue"]*6, markercolor=["blue"]*6, fontsize=16)
fig.show()

In [ ]:
# Plot deviatoric stress average vs time
time_broadcasted = np.tile(time_steps[:, np.newaxis], (1, stress_average.shape[1]))
fig = plot_subplots(time_broadcasted, stress_average_deviatoric, labels_x=['Time']*6, labels_y=['Stress_dev']*6,
                    subplot_titles=['component - 11', 'component - 22', 'component - 33', 'component - 12', 'component - 13', 'component - 23'],
                    title='Deviatoric stress average vs time',
                    nrows=2, ncols=3, linewidth=1, markersize=4, linecolor=["blue"]*6, markercolor=["blue"]*6, fontsize=16)
fig.show()

In [ ]:
fig = plot_subplots(np.column_stack((strain_average[:, 0], strain_average[:, 0])), 
                    np.column_stack((stress_average[:, 0], stress_average_deviatoric[:, 0])), 
                    labels_x=['Strain_11']*2, labels_y=['Stress_11', 'Stress_dev_11'],
                    subplot_titles=['Stress_11 vs Strain_11', 'Stress_dev_11 vs Strain_11'],
                    nrows=1, ncols=2, linewidth=1, markersize=4, linecolor=["blue"]*2, markercolor=["blue"]*2, fontsize=16)
fig.show()

## Step 6: Exporting Data to XDMF for Visualization
The final step in the workflow is to export the HDF5 data to an [XDMF](https://www.xdmf.org/index.php/XDMF_Model_and_Format) file, which is a format that can be easily read by visualization tools like [ParaView](https://www.paraview.org/). This is particularly useful when you want to visualize the spatial and temporal evolution of fields such as stress, strain, or any other tensor quantities in 3D. 

The `write_xdmf` function converts the data stored in an HDF5 file into an XDMF format. This allows you to map the structured data in the HDF5 file to a grid-based format that can be rendered in 3D, providing insights into the distribution and evolution of physical quantities across the microstructure over time.

In addition to this, the `h52xdmf.py` script can also be used as a command-line tool. This allows for flexible conversion of HDF5 files to XDMF format. For further details run `python h52xdmf.py --help`. This will display detailed usage information, including all available options and arguments.


### Visualization in ParaView
Once the XDMF file is generated, it can be loaded into ParaView. ParaView will recognize the spatial and temporal structure of the data, allowing you to explore the 3D microstructure interactively. You can create slices, isosurfaces, and animations to visualize how quantities like stress and strain evolve over time.


In [ ]:
write_xdmf(h5_filepath=file_path, xdmf_filepath='test_results.xdmf',
           microstructure_length=[1,1,1], time_series=True, time_keyword='time_step', verbose=True)

## Summary

The FANS Dashboard provides a streamlined workflow for interpreting and visualizing FANS simulation results. By following the steps outlined above, you can effectively explore your data, perform critical post-processing, and prepare your results for detailed visualization in ParaView.

For more advanced usage and customizations, explore the full set of utilities available in the FANS Dashboard modules.

